In [1]:
:load ./learning-2022/files/LectureNotes/Sections/interpreter/Parsing

{-# LANGUAGE TypeSynonymInstances #-}
{-# LANGUAGE FlexibleInstances    #-}

### Syntax

In [2]:
data Register = ACC
              | NIL
           -- | ANY
           -- | LAST
              | IN
              | OUT
           -- non-addressable:
              | BAK
              | IP
              deriving (Show, Read, Eq)

data Operand = Reg Register
             | Con Int
             deriving (Read)

type Condition = Int -> Bool
type Label = String

data Operation = MOV Operand Register
               | SWP
            -- | SAV -> MOV ACC BAK
            -- | NOP -> ADD NIL
               | ADD Operand
               | SUB Operand
               | NEG
               | Jmp Condition Label     -- for parsing only
               | JRO Condition Operand
            -- | JMP | JEZ | JNZ | JGZ | JLZ
               | HCF
               deriving (Show, Read)



-- instances

instance Show Operand where
  show (Reg x) = show x
  show (Con x) = show x

instance Show Condition where
  show c = case map c [(-1)..1] of
             [True, True, True ] -> "__"
             [False,True, False] -> "EZ"
             [True, False,True ] -> "NZ"
             [False,False,True ] -> "GZ"
             [True, False,False] -> "LZ"

instance Read Condition where
  readsPrec _ _ = []

### Parsing

In [3]:
import Text.Read
import Data.Maybe
import Control.Monad

import Parsing

In [4]:
goodchar :: Parser Char
goodchar = sat (`notElem` " :#\n")

word :: Parser String
word = token $ many goodchar

register :: Parser Register
register = do xs <- word
              guard $ isJust (readMaybe xs :: Maybe Register)
              return (read xs :: Register)

operand :: Parser Operand
operand =  Reg <$> register
       <|> Con <$> integer

operation :: Parser Operation
operation =  do symbol "MOV"
                o <- operand
                MOV o <$> register
         <|> do symbol "SAV"
                return (MOV (Reg ACC) BAK)
         <|> do symbol "NOP" 
                return (ADD (Reg NIL))
         <|> do symbol "ADD"
                ADD <$> operand
         <|> do symbol "SUB"
                SUB <$> operand
         <|> do symbol "JMP"
                Jmp (const True) <$> word
         <|> do symbol "JEZ"
                Jmp (== 0)       <$> word
         <|> do symbol "JNZ"
                Jmp (/= 0)       <$> word
         <|> do symbol "JGZ"
                Jmp (>  0)       <$> word
         <|> do symbol "JLZ"
                Jmp (<  0)       <$> word
         <|> do symbol "JRO"
                JRO (const True) <$> operand
         <|> do xs <- word
                guard $ isJust (readMaybe xs :: Maybe Operation)
                return (read xs :: Operation)
                
label :: Parser Label
label = token $
          do w <- many goodchar
             c <- sat (==':')
             return w

node :: Parser [Either Operation Label]
node = many $
        Left  <$> operation
    <|> Right <$> label

process :: [Either Operation Label] -> (Label -> Int) -> [Operation] -> [Operation]
process [] f os = p os
  where p [] = []
        p xs = p (init xs) ++ [q (last xs)]
          where q (Jmp c s) = JRO c (Con (f s - length (init xs)))
                q x         = x
process (Left  o : es) f n = process es f (n ++ [o])
process (Right l : es) f n = process es f' n
  where f' l' | l' == l   = if null es then 0 else length n
              | otherwise = f l'

parseNode :: String -> [Operation]
parseNode xs = case parse node xs of
                 [(p , [])] -> process p (const undefined) []
                 [(_ , s)]  -> error ("syntax: unparsed string " ++ s)
                 _          -> error "syntax: failed to parse"

### Interpreting

In [5]:
type State = Register -> Int

data Node = Node [Operation] State

instance Show Node where
  --show n = show (current n) ++ " | " ++ show (get ACC n) ++ " (" ++ show (get BAK n) ++ ") " ++ show (get IN n) ++ " " ++ show (get OUT n) ++ "\n"
  show (Node os _) = show os

makeNode :: String -> Node
makeNode xs = Node (parseNode xs) (const 0)

clamp :: Int -> Int -> Int -> Int
clamp l h x = max l $ min h x

update :: Register -> Int -> Node -> Node
update NIL _ n = n
update r   x (Node os s) = Node os s'
  where s' r' | r' == r   = clamp (-999) 999 x
              | otherwise = s r'

get :: Register -> Node -> Int
get r (Node os s) = s r

len :: Node -> Int
len (Node os _) = length os

current :: Node -> Operation
current (Node os s) = os !! s IP

inc :: Node -> Node
inc n = update IP ((get IP n + 1) `mod` len n) n

jump :: Int -> Node -> Node
jump i n = update IP (clamp 0 (len n-1) i) n

eval :: Operand -> Node -> Int
eval (Reg r) s = get r s
eval (Con x) _ = x

exec :: Node -> Node
exec n = e (current n) n
  where
    e :: Operation -> Node -> Node
    e (MOV o r) n = update r (eval o n) $ inc n
    e  SWP      n = update BAK (get ACC n) $ update ACC (get BAK n) $ inc n
    e (ADD o)   n = update ACC (get ACC n + eval o n) $ inc n
    e (SUB o)   n = update ACC (get ACC n - eval o n) $ inc n
    e  NEG      n = update ACC (negate $ get ACC n) $ inc n
    e (JRO c o) n = if c $ get ACC n then jump (get IP n + eval o n) n else inc n
    e  HCF      n = error "halted and caught fire"

run :: Node -> Int -> Int
run n i = get OUT $ iterate exec (update IN i n) !! 1000

### Testing

In [6]:
example1 = 
  unlines [
    "  MOV IN  ACC",
    "  SAV        ",
    "  MOV 1   ACC",
    "             ",
    "L:SWP        ",
    "  JEZ E      ",
    "  JLZ F      ",
    "  SUB 1      ",
    "  SWP        ",
    "  ADD ACC    ",
    "  JMP L      ",
    "             ",
    "F:HCF        ",
    "             ",
    "E:SWP        ",
    "  MOV ACC OUT"]

putStr example1

  MOV IN  ACC
  SAV        
  MOV 1   ACC
             
L:SWP        
  JEZ E      
  JLZ F      
  SUB 1      
  SWP        
  ADD ACC    
  JMP L      
             
F:HCF        
             
E:SWP        
  MOV ACC OUT

In [7]:
parsed1 = makeNode example1
parsed1

[MOV IN ACC,MOV ACC BAK,MOV 1 ACC,SWP,JRO EZ 7,JRO LZ 5,SUB 1,SWP,ADD ACC,JRO __ -6,HCF,SWP,MOV ACC OUT]

In [8]:
map (run parsed1) [0..11]

[1,2,4,8,16,32,64,128,256,512,999,999]

In [9]:
run parsed1 (-1)

: 

In [10]:
example2 =
  unlines [
    "  MOV 20 ACC ",
    "  JRO IN     ",
    "  JMP E      ",
    "             ",
    "2:SUB 10     ",
    "  JMP E      ",
    "             ",
    "4:SUB 1      ",
    "  JMP E      ",
    "             ",
    "6:ADD 1      ",
    "  JMP E      ",
    "             ",
    "8:ADD 10     ",
    "             ",
    "E:MOV ACC OUT"]
           
putStr example2

  MOV 20 ACC 
  JRO IN     
  JMP E      
             
2:SUB 10     
  JMP E      
             
4:SUB 1      
  JMP E      
             
6:ADD 1      
  JMP E      
             
8:ADD 10     
             
E:MOV ACC OUT

In [11]:
parsed2 = makeNode example2
parsed2

[MOV 20 ACC,JRO __ IN,JRO __ 8,SUB 10,JRO __ 6,SUB 1,JRO __ 4,ADD 1,JRO __ 2,ADD 10,MOV ACC OUT]

In [12]:
map (run parsed2) [1..11]

[20,10,20,19,20,21,20,30,20,20,20]